## Часть 1. Цель задания

В этом практическом задании проводится тематическое моделирование текстов с помощью алгоритма LDA (Latent Dirichlet Allocation). Цель — определить скрытые темы в корпусе документов, связанных с обороной, международными военными учениями, кибербезопасностью и технологическим развитием в армии.

Используем библиотеки:
- `nltk` для предобработки текста
- `gensim` для построения тематической модели
- `pyLDAvis` для визуализации результатов


In [1]:
# !pip install gensim nltk pyLDAvis

import nltk
import gensim
from gensim import corpora
from gensim.models import LdaModel
import pyLDAvis
import pyLDAvis.gensim_models
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')


/Users/aikei/PycharmProjects/IRIE_practice-10/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
[nltk_data] Downloading package punkt to /Users/aikei/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/aikei/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Часть 2. Подготовка документов

Ниже представлены документы, охватывающие темы дронов, кибербезопасности, международных договоров, совместных учений и новых разработок.


In [2]:
documents = [
    "The Pentagon has approved a new budget to expand drone surveillance in the Indo-Pacific region.",
    "Russia and China are investing in cyber and electronic warfare technologies.",
    "France and Germany signed a bilateral defense treaty focused on joint drone manufacturing.",
    "The U.S. Navy conducted joint military exercises with Japan and South Korea in the Pacific Ocean.",
    "Lockheed Martin is developing a next-generation missile defense system for the U.S. Army."
]


## Часть 3. Предобработка текста

Выполняем токенизацию, удаление стоп-слов и очистку от пунктуации.


In [3]:
stop_words = set(stopwords.words("english"))

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    return tokens

processed_docs = [preprocess_text(doc) for doc in documents]
print(processed_docs)


[['pentagon', 'approved', 'new', 'budget', 'expand', 'drone', 'surveillance', 'region'], ['russia', 'china', 'investing', 'cyber', 'electronic', 'warfare', 'technologies'], ['france', 'germany', 'signed', 'bilateral', 'defense', 'treaty', 'focused', 'joint', 'drone', 'manufacturing'], ['navy', 'conducted', 'joint', 'military', 'exercises', 'japan', 'south', 'korea', 'pacific', 'ocean'], ['lockheed', 'martin', 'developing', 'missile', 'defense', 'system', 'army']]


## Часть 4. Создание словаря и корпуса

Для обучения LDA создаём словарь и корпус на основе предобработанных текстов.


In [4]:
dictionary = corpora.Dictionary(processed_docs)
corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

print(dictionary.token2id)
print(corpus)


{'approved': 0, 'budget': 1, 'drone': 2, 'expand': 3, 'new': 4, 'pentagon': 5, 'region': 6, 'surveillance': 7, 'china': 8, 'cyber': 9, 'electronic': 10, 'investing': 11, 'russia': 12, 'technologies': 13, 'warfare': 14, 'bilateral': 15, 'defense': 16, 'focused': 17, 'france': 18, 'germany': 19, 'joint': 20, 'manufacturing': 21, 'signed': 22, 'treaty': 23, 'conducted': 24, 'exercises': 25, 'japan': 26, 'korea': 27, 'military': 28, 'navy': 29, 'ocean': 30, 'pacific': 31, 'south': 32, 'army': 33, 'developing': 34, 'lockheed': 35, 'martin': 36, 'missile': 37, 'system': 38}
[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)], [(8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1)], [(2, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1)], [(20, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1)], [(16, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1)]]


## Часть 5. Тематическое моделирование

Используем модель LDA для выявления скрытых тем.


In [5]:
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=3, passes=10, random_state=42)

for idx, topic in lda_model.print_topics(num_words=5):
    print(f"Тема {idx}: {topic}")


Тема 0: 0.058*"joint" + 0.058*"korea" + 0.058*"navy" + 0.058*"conducted" + 0.058*"exercises"
Тема 1: 0.064*"drone" + 0.063*"region" + 0.063*"budget" + 0.063*"expand" + 0.063*"surveillance"
Тема 2: 0.063*"defense" + 0.036*"manufacturing" + 0.036*"france" + 0.036*"signed" + 0.036*"germany"


## Часть 6. Определение тем для каждого документа

Теперь посмотрим, какие темы наиболее характерны для каждого документа.


In [6]:
for i, doc_bow in enumerate(corpus):
    topics = lda_model.get_document_topics(doc_bow)
    print(f"Документ {i+1}: {topics}")


Документ 1: [(0, 0.03727333), (1, 0.9252026), (2, 0.03752409)]
Документ 2: [(0, 0.042137858), (1, 0.042187236), (2, 0.91567487)]
Документ 3: [(0, 0.03135371), (1, 0.031474322), (2, 0.937172)]
Документ 4: [(0, 0.93879277), (1, 0.030530948), (2, 0.030676316)]
Документ 5: [(0, 0.042102072), (1, 0.042157795), (2, 0.91574013)]


## Часть 7. Визуализация тем с помощью pyLDAvis

Используем интерактивную визуализацию для анализа распределения тем.


In [7]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(vis)
